# Test MISA CRM API Endpoints

Notebook này dùng để kiểm tra việc lấy dữ liệu từ các API endpoint của MISA CRM. 

**Yêu cầu:** Cần điền đầy đủ `MISA_CRM_CLIENT_ID` và `MISA_CRM_CLIENT_SECRET` trong file `.env` trước khi chạy.

In [3]:
import requests
import os
import json
import pandas as pd
from dotenv import load_dotenv

# Tải các biến môi trường từ file .env
load_dotenv()

# Lấy thông tin từ biến môi trường
MISA_CLIENT_ID = os.getenv('MISA_CRM_CLIENT_ID')
MISA_CLIENT_SECRET = os.getenv('MISA_CRM_CLIENT_SECRET')
BASE_URL = 'https://crmconnect.misa.vn'

print(f"Client ID: {MISA_CLIENT_ID}")
print(f"Client Secret is {'set' if MISA_CLIENT_SECRET else 'not set'}")

Client ID: FACOLOS
Client Secret is set


## 1. Lấy Access Token

In [4]:
def get_misa_access_token():
    """Hàm để lấy access token từ MISA CRM API."""
    token_url = f"{BASE_URL}/api/v2/Account"
    
    if not MISA_CLIENT_ID or not MISA_CLIENT_SECRET:
        print('Lỗi: Vui lòng điền MISA_CRM_CLIENT_ID và MISA_CRM_CLIENT_SECRET trong file .env')
        return None
    
    payload = {
        'client_id': MISA_CLIENT_ID,
        'client_secret': MISA_CLIENT_SECRET
    }
    headers = {'Content-Type': 'application/json'}
    
    try:
        response = requests.post(token_url, json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        
        result = response.json()
        if result.get('success') and result.get('data'):
            access_token = result['data']
            print('Lấy access token thành công!')
            return access_token
        else:
            print(f"Lấy token thất bại: {result.get('error_message', result)}")
            return None
    except requests.exceptions.RequestException as e:
        print(f'Lỗi kết nối khi lấy token: {e}')
        return None

access_token = get_misa_access_token()
# In một phần token để xác nhận
if access_token:
    print(f'Access Token (Full): {access_token}')

Lấy access token thành công!
Access Token (Full): eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJwYXlsb2FkZGF0YSI6Ik9NdVc2dytKL0o4dWN0ZUVwQ2x4dkw0aHFzM2VzekVna3J6Yms3ZmxCUVZWYnBwTVZDczNtMWxRRC9DcGRHNnhJTnkrL2tnZ3N1Y3hjdWRwZU9rTDRENTJKakxIN1lRM2IyV0lKVW54ZnE3NWc4RGpDd3FTL1R3UW1SZGV4dmdEOENGL08wVWRRbisvRVNyU1Y5cEdpSGUyR09jdTZ5dEd2MXNMUVEwOFRsRDZFa0NxbE1BL3ZiWGlxVDRVdW9CUnNTYkhYWVFaaTRpYkNaS3UwT3RVY3YzbW9tNlhxeElVTlBGZFlGOWRxdnp3aHIvWXhpR1pZUjRkakZ0a21aNVZPYnlMcmFtMzczb3FWNU5MQnJ1MFVmVmVBZ2FVR0JId1RKR2RCOE1oKy9nQkpsN0VqTml1dHpUUDlncnZlYzRpOHRLOHFENzR0K3o2MDRQbDRrOW5xZz09IiwiZXhwIjoxNzYwNDY4OTA4LCJpc3MiOiJNSVNBIiwiYXVkIjoiQU1JU0NSTTIifQ.Pw_u6qJIWWjbM6kce3TWQBB0sDSZnElmNRf3udm9gL0


## 2. Hàm Test Endpoint

In [3]:
def test_endpoint(endpoint_path, access_token, params=None):
    """Hàm chung để test một endpoint với phân trang."""
    if not access_token:
        print('Không có access token, không thể thực hiện request.')
        return
    
    url = f"{BASE_URL}{endpoint_path}"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Clientid': MISA_CLIENT_ID
    }
    
    # Mặc định lấy trang đầu tiên, 5 bản ghi
    default_params = {'page': 0, 'pageSize': 5}
    if params:
        default_params.update(params)
    
    print(f'--- Đang gọi đến: {url} với params: {default_params} ---')
    
    try:
        response = requests.get(url, headers=headers, params=default_params, timeout=30)
        response.raise_for_status()
        
        result = response.json()
        print(f'Status Code: {response.status_code}')
        print('Response JSON:')
        print(json.dumps(result, indent=2, ensure_ascii=False))
        
        if result.get('success') and isinstance(result.get('data'), list):
            df = pd.DataFrame(result['data'])
            print('Hiển thị dữ liệu dưới dạng DataFrame:')
            display(df)
        else:
            print('Không có dữ liệu hoặc cấu trúc response không đúng.')
            
    except requests.exceptions.RequestException as e:
        print(f'Lỗi request: {e}')

## 3. Test Các Endpoint Cụ Thể

In [4]:
# Endpoint 1: Customers
test_endpoint('/api/v2/Customers', access_token)

--- Đang gọi đến: https://crmconnect.misa.vn/api/v2/Customers với params: {'page': 0, 'pageSize': 5} ---
Status Code: 200
Response JSON:
{
  "success": true,
  "code": 200,
  "data": [
    {
      "id": 3062,
      "owner_name": "",
      "account_short_name": null,
      "fax": null,
      "parent_account_name": null,
      "account_type": null,
      "sector_name": null,
      "no_of_employee_name": null,
      "bank_account": null,
      "celebrate_date": null,
      "lead_source": null,
      "created_date": "2025-09-11T09:18:37.000+07:00",
      "created_by": "",
      "account_name": "Đỗ Mạnh Tuyền",
      "rating": null,
      "office_email": null,
      "website": null,
      "account_number": "KH001534",
      "business_type": null,
      "industry": null,
      "annual_revenue": null,
      "budget_code": null,
      "bank_name": null,
      "customer_since_date": null,
      "is_public": null,
      "modified_date": "2025-09-26T17:48:35.000+07:00",
      "modified_by": "Nguy

,id,owner_name,account_short_name,fax,parent_account_name,account_type,sector_name,no_of_employee_name,bank_account,celebrate_date,...,is_distributor,purchase_date_first,is_portal_access,portal_username,shipping_long,shipping_lat,billing_long,billing_lat,custom_field13,custom_field14
0,3062,,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,3387,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,3388,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,3389,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,3384,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [5]:
# Endpoint 2: SaleOrders
test_endpoint('/api/v2/SaleOrders', access_token)

--- Đang gọi đến: https://crmconnect.misa.vn/api/v2/SaleOrders với params: {'page': 0, 'pageSize': 5} ---
Status Code: 200
Response JSON:
{
  "success": true,
  "code": 200,
  "data": [
    {
      "id": 5279,
      "sale_order_no": "DH25092558",
      "campaign_name": null,
      "book_date": "2025-09-26T00:00:00.000+07:00",
      "sale_order_amount": 163350000.1,
      "deadline_date": null,
      "quote_name": null,
      "status": "Chưa thực hiện",
      "exchange_rate": 1.0,
      "sale_order_name": "Đơn hàng bán cho Hà Tuấn Sơn",
      "sale_order_date": "2025-09-26T00:00:00.000+07:00",
      "account_name": "Hà Tuấn Sơn",
      "contact_name": null,
      "due_date": "2025-09-26T00:00:00.000+07:00",
      "delivery_status": "Chưa giao hàng",
      "sale_order_type": "Bán mới",
      "opportunity_name": null,
      "revenue_status": "Bản nháp",
      "currency_type": "VND",
      "description": null,
      "balance_receipt_amount": 163350000.1,
      "pay_status": "Chưa thanh toá

,id,sale_order_no,campaign_name,book_date,sale_order_amount,deadline_date,quote_name,status,exchange_rate,sale_order_name,...,production_date,production_rejection_reason,produced_quantity_summary,account_code,contact_code,opportunity_code,quote_code,employee_code,is_deleted,sale_order_product_mappings
0,5279,DH25092558,None,2025-09-26T00:00:00.000+07:00,1.633500e+08,None,None,Chưa thực hiện,1.0,Đơn hàng bán cho Hà Tuấn Sơn,...,None,None,NaN,NPP31,None,None,None,NV12,False,"[{'id': 19482, 'to_currency_oc': 41666666.7, '..."
1,5278,DH25092557,None,2025-09-26T00:00:00.000+07:00,8.662500e+07,None,None,Chưa thực hiện,1.0,Đơn hàng bán cho CÔNG TY TNHH THƯƠNG MẠI VÀ DỊ...,...,None,None,0.0,NPP01-3,None,None,None,NV09,False,"[{'id': 19472, 'to_currency_oc': 20833333.35, ..."
2,5102,DH0046606,None,2025-09-22T00:00:00.000+07:00,1.620000e+06,None,None,Chưa thực hiện,1.0,Đơn hàng tài trợ Giải thi đấu môn Pickleball P...,...,None,None,0.0,KH00001191,None,None,None,KIM,False,"[{'id': 18877, 'to_currency_oc': 1500000.0, 'd..."
3,5129,DH0046710,None,2025-09-24T00:00:00.000+07:00,7.830000e+07,None,None,Đã thực hiện,1.0,Đơn hàng bán cho CÔNG TY TNHH TLT SPORTS,...,None,None,0.0,NPPCL01,None,None,None,NV06,False,"[{'id': 18942, 'to_currency_oc': 66666668.0, '..."
4,5171,DH0046827,None,2025-09-24T00:00:00.000+07:00,8.550000e+06,None,None,Đã thực hiện,1.0,Đơn hàng bán cho CÔNG TY TNHH TLT SPORTS + tặn...,...,None,None,0.0,NPPCL01,None,None,None,NV06,False,"[{'id': 19128, 'to_currency_oc': 8333333.5, 'd..."


In [4]:
# Endpoint 3: Contacts
test_endpoint('/api/v2/Contacts', access_token)

--- Đang gọi đến: https://crmconnect.misa.vn/api/v2/Contacts với params: {'page': 0, 'pageSize': 5} ---
Status Code: 200
Response JSON:
{
  "success": true,
  "code": 200,
  "data": [
    {
      "id": 6901,
      "contact_code": "LH0000217",
      "salutation": null,
      "last_name": "Jun Hwa",
      "first_name": "Joe Lew ",
      "contact_name": "Joe Lew  Jun Hwa",
      "title": null,
      "department": null,
      "account_name": "DINKit Store Sdn Bhd",
      "mobile": "+(60)173997933",
      "other_phone": null,
      "office_email": null,
      "email": null,
      "office_tel": null,
      "lead_source": null,
      "account_type": null,
      "customer_since_date": null,
      "description": null,
      "email_opt_out": false,
      "phone_opt_out": false,
      "mailing_country": "Malaysia",
      "mailing_province": null,
      "mailing_district": null,
      "mailing_ward": null,
      "mailing_street": null,
      "mailing_zip": "MY",
      "mailing_address": "1, Jln SS

,id,contact_code,salutation,last_name,first_name,contact_name,title,department,account_name,mobile,...,last_interaction_date,last_visit_date,number_days_not_interacted,last_call_date,related_users,mailing_long,mailing_lat,shipping_long,shipping_lat,account_code
0,6901,LH0000217,None,Jun Hwa,Joe Lew,Joe Lew Jun Hwa,None,None,DINKit Store Sdn Bhd,+(60)173997933,...,None,None,None,None,None,None,None,None,None,KH00000336
1,6912,LH0000228,None,Anh Tùng,None,Anh Tùng,None,None,Wouter Cornelis van der Wiel,0938 242 756,...,None,None,None,None,None,None,None,None,None,KH00001299
2,6911,LH0000227,None,Mr. Tony Truong,None,Mr. Tony Truong,None,None,Mr Tony Truong,+8613922274686,...,None,None,None,None,None,None,None,None,None,KH00001279
3,6910,LH0000226,None,Phương,Vũ Thu,Vũ Thu Phương,None,None,None,0963457295,...,None,None,None,None,None,None,None,None,None,None
4,6909,LH0000225,None,Nguyễn Chung Nam,None,Nguyễn Chung Nam,None,None,Nguyễn Chung Nam,0373323654,...,None,None,None,None,None,None,None,None,None,KH00001162


In [ ]:
# Endpoint 4: Stocks
# Endpoint này không có phân trang, nên không cần params
test_endpoint('/api/v2/Stocks', access_token, params={})

In [5]:
# Endpoint 5: Products (Giả định, vì không có trong tài liệu v2)
# Chúng ta sẽ thử gọi đến /api/v2/Products để xem kết quả
print('Lưu ý: Endpoint /api/v2/Products không được liệt kê trong tài liệu Swagger v2, kết quả có thể không thành công.')
test_endpoint('/api/v2/Products', access_token)

Lưu ý: Endpoint /api/v2/Products không được liệt kê trong tài liệu Swagger v2, kết quả có thể không thành công.
--- Đang gọi đến: https://crmconnect.misa.vn/api/v2/Products với params: {'page': 0, 'pageSize': 5} ---
Status Code: 200
Response JSON:
{
  "success": true,
  "code": 200,
  "data": [
    {
      "id": 15464,
      "owner_name": null,
      "product_code": "TUI_XACH",
      "product_category": "Hàng hóa",
      "usage_unit": "Cái",
      "product_name": "Túi xách cỏ bàng in theo yêu cầu",
      "inactive": false,
      "unit_price": 0.0,
      "tax": null,
      "description": null,
      "created_date": "2025-10-02T15:43:04.000+07:00",
      "created_by": "SME",
      "modified_date": "2025-10-02T15:43:04.000+07:00",
      "modified_by": "SME",
      "tag": null,
      "purchased_price": 0.0,
      "price_after_tax": false,
      "is_public": null,
      "form_layout": "Mẫu tiêu chuẩn",
      "organization_unit_name": "CÔNG TY CỔ PHẦN THỂ THAO FACOLOS",
      "is_follow_seri

,id,owner_name,product_code,product_category,usage_unit,product_name,inactive,unit_price,tax,description,...,unit_cost,sale_description,quantity_formula,product_properties,default_stock,warranty_period,source,is_set_product,warranty_description,avatar
0,15464,None,TUI_XACH,Hàng hóa,Cái,Túi xách cỏ bàng in theo yêu cầu,False,0.0,None,None,...,None,Túi xách cỏ bàng in theo yêu cầu,None,Hàng hóa,None,None,None,False,None,None
1,15463,None,MAYMASSA2,Hàng hóa,Chiếc,Máy massage dạng ghế Poongsan MCP-106,False,0.0,None,None,...,None,Máy massage dạng ghế Poongsan MCP-106,None,Hàng hóa,None,None,None,False,None,None
2,15462,None,In UV mặt vợt,Dịch vụ,Lần,In UV mặt vợt,False,0.0,None,None,...,None,In UV mặt vợt,None,Dịch vụ,None,None,None,False,None,None
3,15461,None,MAYMASSA,Hàng hóa,Cái,MÁY MASSAGE CỔ VAI GÁY PHILIPS PPM3522,False,0.0,None,None,...,None,MÁY MASSAGE CỔ VAI GÁY PHILIPS PPM3522,None,Hàng hóa,None,None,None,False,None,None
4,15460,None,BAOHANH,Hàng hóa,Cái,BẢO HÀNH 1 ĐỔI 1 VIP - 12 THÁNG - PHỤ KIỆN C...,False,0.0,None,None,...,None,BẢO HÀNH 1 ĐỔI 1 VIP - 12 THÁNG - PHỤ KIỆN C...,None,Hàng hóa,None,None,None,False,None,None
